<a href="https://colab.research.google.com/github/AllanKamimura/buzzleixo/blob/master/buzzleixo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate \
    https://github.com/AllanKamimura/buzzleixo/blob/master/Train.zip?raw=true \
    -O /tmp/Train.zip

In [ ]:
import os
import zipfile

local_zip = '/tmp/Train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Directory with our training buzz pictures
train_buzz_dir = os.path.join('/tmp/Train/Buzz')

# Directory with our training juju pictures
train_juju_dir = os.path.join('/tmp/Train/Juju')

train_buzz_names = os.listdir(train_buzz_dir)
print(train_buzz_names[:10])

train_juju_names = os.listdir(train_juju_dir)
print(train_juju_names[:10])

print('total training buzz images:', len(os.listdir(train_buzz_dir)))
print('total training juju images:', len(os.listdir(train_juju_dir)))

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32 , (3,3), activation = "relu", input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")])
model.summary()


In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss = "binary_crossentropy", 
    optimizer = RMSprop(lr = 0.0001), 
    metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255

train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 45,
    width_shift_range = .15,
    height_shift_range = .15,
    horizontal_flip = True,
    zoom_range = 0.5)


# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/Train',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size= 5,
        shuffle = True,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode = 'binary')

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 10,
    epochs = 25,
    verbose = 1)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " É UM ABACATE")
    %pylab inline
    img = mpimg.imread('/tmp/aleixo.jpg')
    imgplot = plt.imshow(img)
    plt.show()
  else:
    print(fn + " É UM BUZZLATIR")
    %pylab inline
    img = mpimg.imread('/tmp/buzz.jpg')
    imgplot = plt.imshow(img)
    plt.show()